# Text Mining 

## 1. Préprocessing

In [ ]:
import pandas as pd

df = pd.read_parquet("flickr_data_clustered.parquet")
print(f"Données chargées : {len(df)} photos")

### Définition des stopwords

On définit une liste de stopwords française et anglaise minimale, en gardant uniquement les mots non-pertinents pour identifier les lieux touristiques de Lyon.

In [ ]:
stopwords = {
    
}

print(f"Nombre total de stopwords : {len(stopwords)}")

### Fonction de nettoyage du texte

On crée une fonction qui :
1. Concatène title et tags
2. Met en minuscules
3. Supprime les accents (é → e, ç → c)
4. Supprime la ponctuation et caractères spéciaux
5. Filtre les stopwords et mots trop courts

In [ ]:
import re
import unicodedata

def clean_text(title, tags):
    """
    Nettoie et combine title et tags en une liste de mots pertinents.
    
    Args:
        title: Titre de la photo (str ou NaN)
        tags: Tags de la photo (str ou NaN)
    
    Returns:
        Liste de mots nettoyés
    """
    # Combiner title et tags
    text = ""
    if isinstance(title, str):
        text += title + " "
    if isinstance(tags, str):
        text += tags
    
    if not text.strip():
        return []
    
    # 1. Minuscules
    text = text.lower()
    
    # 2. Supprimer accents et caractères spéciaux Unicode
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    
    # 3. Ne garder que lettres et espaces (supprime [, ], {, }, etc.)
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # 4. Split et filtrer
    words = text.split()
    words = [w for w in words if w not in stopwords and len(w) > 2]
    
    return words

# Test de la fonction
test_title = "[Lyon] Basilique de Fourvière - Été 2024"
test_tags = "france, architecture, église, photo"
print("Test de nettoyage :")
print(f"Input : '{test_title}' + '{test_tags}'")
print(f"Output : {clean_text(test_title, test_tags)}")

### Application du nettoyage

On applique la fonction de nettoyage sur toutes les photos pour créer une colonne unique `cleaned_text`.

In [ ]:
# Appliquer le nettoyage sur title + tags
df['cleaned_text'] = df.apply(
    lambda row: clean_text(row['title'], row['tags']), 
    axis=1
)

# Statistiques sur le nettoyage
total_photos = len(df)
photos_avec_mots = (df['cleaned_text'].str.len() > 0).sum()
photos_sans_mots = total_photos - photos_avec_mots

print(f"✓ Nettoyage terminé")
print(f"  - Photos avec mots : {photos_avec_mots} ({photos_avec_mots/total_photos*100:.1f}%)")
print(f"  - Photos sans mots : {photos_sans_mots} ({photos_sans_mots/total_photos*100:.1f}%)")
print(f"  - Nombre moyen de mots/photo : {df['cleaned_text'].str.len().mean():.1f}")

### Vérification du résultat

On affiche quelques exemples pour vérifier que le nettoyage fonctionne correctement.

In [ ]:
# Afficher quelques exemples
sample = df[df['cleaned_text'].str.len() > 0].sample(n=10, random_state=42)

print("Exemples de textes nettoyés :\n")
for idx, row in sample.iterrows():
    print(f"Title original : {row['title'][:80]}...")
    print(f"Tags originaux : {row['tags'][:80] if isinstance(row['tags'], str) else 'N/A'}...")
    print(f"Mots nettoyés  : {row['cleaned_text'][:10]}")  # 10 premiers mots
    print(f"Cluster        : {row['cluster_hdbscan']}")
    print("-" * 80)

## 2. Méthode 1 : Mot le plus fréquent par cluster

Ensuite on on cherche le mot qui correspond à chaque cluster
df : le dataframe 
cluster_kmeans : le nom de la colonne qui contient le num de cluster auquel appartient la ligne
texte_cols : les colonnes où y a les textes qu'on va utiliser pour identifier les mots les plus fréquent 
top_k : nb de mots à garder = 1 puisque on cherche un seul mots par cluster

In [ ]:
df

In [ ]:
from collections import Counter

def cluster_titles(
    df,
    cluster_hdbscan="cluster_hdbscan",
    text_cols=("cleaned_title", "cleaned_tags"),
    top_k=1
):
    """
    Retourne un titre (mot-clé) par cluster basé sur les mots les plus fréquents
    """
    cluster_labels = {}

    for cluster_id in sorted(df[cluster_hdbscan].unique()):
        if cluster_id == -1:
            continue  # on ignore le bruit

        # sous-dataframe du cluster (seulement les ligne de ce cluster)
        dff = df[df[cluster_hdbscan] == cluster_id]

        # concaténation des textes de toutes les lignes
        all_words = []
        #on parcours les deux colonnes de texte
        for col in text_cols:
            texts = dff[col].dropna()
            for t in texts:
                all_words.extend(t)
        # si la liste est vide donc pas de mots
        if not all_words:
            cluster_labels[cluster_id] = "unknown"
            continue

        # comptage des mots
        counts = Counter(all_words)

        # mots les plus fréquents
        top_words = [w for w, _ in counts.most_common(top_k)]

        # Retourner le premier mot (string) au lieu d'une liste
        cluster_labels[cluster_id] = top_words[0] if top_words else "unknown"

    return cluster_labels

df['cluster_name'] = df['cluster_hdbscan'].map(
    cluster_titles(df, top_k=1)
).fillna("unknown")

In [ ]:
df 

In [ ]:
# Vérifier ce que retourne la fonction
print("Premiers cluster_name:")
print(df[['cluster_hdbscan', 'cluster_name']].head(20))
print("\nTypes:")
print(df['cluster_name'].dtype)
print("\nValeurs uniques (10 premières):")
print(df['cluster_name'].unique()[:10])

In [ ]:
# Afficher TOUS les clusters avec leurs titres
cluster_summary = df[df['cluster_hdbscan'] != -1].groupby('cluster_hdbscan').agg({
    'cluster_name': 'first',
    'id': 'count'
}).rename(columns={'id': 'nb_photos'}).sort_values('nb_photos', ascending=False)

print(f"Liste complète des {len(cluster_summary)} clusters avec leurs titres:\n")
print(cluster_summary.to_string())

# Ou en DataFrame pour mieux voir
cluster_summary

In [ ]:
# Nombre de clusters HDBSCAN
n_clusters = len(df[df['cluster_hdbscan'] != -1]['cluster_hdbscan'].unique())
n_bruit = len(df[df['cluster_hdbscan'] == -1])
n_total = len(df)

print(f"📊 Statistiques HDBSCAN:")
print(f"  • Nombre de clusters: {n_clusters}")
print(f"  • Points de bruit (-1): {n_bruit} ({n_bruit/n_total*100:.1f}%)")
print(f"  • Points dans des clusters: {n_total - n_bruit} ({(n_total-n_bruit)/n_total*100:.1f}%)")
print(f"\nTaille des 10 plus gros clusters:")
print(df[df['cluster_hdbscan'] != -1]['cluster_hdbscan'].value_counts().head(10))

In [ ]:
import folium

sample = df.sample(n=min(30000, len(df)), random_state=0)

m = folium.Map(
    location=[df["lat"].median(), df["long"].median()],
    zoom_start=12,
    tiles="CartoDB positron"
)

palette = ["red", "blue", "green", "purple", "orange", "darkred", "lightred", 
           "beige", "darkblue", "darkgreen", "cadetblue", "darkpurple", 
           "pink", "lightblue", "lightgreen", "gray", "black", "lightgray"]

for _, r in sample.iterrows():
    cluster = r["cluster_hdbscan"]
    if cluster == -1:
        color = "lightgray"
    else:
        color = palette[cluster % len(palette)]
    
    folium.CircleMarker(
        location=[r["lat"], r["long"]],
        radius=2,
        color=color,
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(
            f"""<b>Keyword:</b> {r["cluster_name"]}<br/>
               <a href="{r["url"]}" target="_blank">Open Flickr</a>""",
            max_width=250
        )
    ).add_to(m)

m

## 3. Méthode 2 : TF-IDF

On tente une autre approche pour décrire chaque cluster : le TF-IDF.

On commence par combiner les listes de mots pour chaque photo

In [ ]:
# Combiner les listes de mots nettoyés en un seul texte par photo
df['combined_text'] = df.apply(
    lambda row: ' '.join(row['cleaned_title_stopwords'] + row['cleaned_tags_stopwords']), 
    axis=1
)

# Vérifier le résultat
df[['cleaned_title_stopwords', 'cleaned_tags_stopwords', 'combined_text', 'cluster_hdbscan']].tail(10)

Pour chaque cluster, on combine les textes de toutes les photos appartenant à ce cluster en un seul document.

In [ ]:
cluster_documents = df.groupby('cluster_hdbscan')['combined_text'].apply(
    lambda texts: ' '.join(texts)
).reset_index()

cluster_documents.columns = ['cluster', 'document']

# Afficher les premiers clusters avec leur taille de texte
cluster_documents['text_length'] = cluster_documents['document'].str.len()
cluster_documents[['cluster', 'text_length']].head(20)

On calcule ensuite le TF-IDF pour chaque mot dans chaque document de cluster.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer le vectorizer
tfidf = TfidfVectorizer(
    max_features=5000,  # Garder les 5000 mots les plus importants
    min_df=2,           # Le mot doit apparaître dans au moins 2 clusters
    max_df=0.8          # Le mot ne doit pas être dans plus de 80% des clusters
)

# Calculer la matrice TF-IDF
tfidf_matrix = tfidf.fit_transform(cluster_documents['document'])

# Voir la forme de la matrice
print(f"Matrice TF-IDF : {tfidf_matrix.shape}")
print(f"(nombre_clusters × nombre_termes)")

On extrait ensuite le top k mots avec les scores TF-IDF les plus élevés pour chaque cluster.

In [ ]:
import pandas as pd

# Récupérer les noms des termes
feature_names = tfidf.get_feature_names_out()

# Fonction pour extraire les top N termes d'un cluster
def get_top_terms(cluster_idx, n=10):
    # Récupérer les scores TF-IDF pour ce cluster
    scores = tfidf_matrix[cluster_idx].toarray().flatten()
    
    # Obtenir les indices des top termes
    top_indices = scores.argsort()[-n:][::-1]
    
    # Créer un dataframe
    top_terms = pd.DataFrame({
        'term': feature_names[top_indices],
        'tfidf_score': scores[top_indices]
    })
    
    return top_terms

# Afficher les top termes du cluster 0
print("Top 10 termes du cluster 0 :")
get_top_terms(0, n=10)

In [ ]:
# Afficher les top 10 termes des 5 premiers clusters
for i in range(min(5, len(cluster_documents))):
    cluster_id = cluster_documents.iloc[i]['cluster']
    print(f"\n{'='*60}")
    print(f"CLUSTER {cluster_id} - Top 10 termes :")
    print(f"{'='*60}")
    display(get_top_terms(i, n=10))

In [ ]:
# Sauvegarder avec la colonne cluster_name
df.to_parquet("flickr_data_clusters_mined.parquet", index=False)